In [112]:

from aocd import get_data, submit
import numpy as np
import sys
import itertools
from collections import defaultdict, Counter
import re
from tqdm.auto import tqdm

day = 24
year = 2022

data = get_data(day=day, year=year)

# data = '\n'.join(re.split(r'[\.|:|\n]', data))
# data = ">>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>"
# print(data)

# face_dim = 50
# data

In [123]:
data = """#.######
#>>.<^<#
#.<..<<#
#>v.><>#
#<^v^^>#
######.#"""

In [152]:
dirs = ((1,0), (0,1), (-1, 0), (0, -1), (0, 0))
dirs_letters = dict(zip(">v<^", dirs))

lines = data.split('\n')
blizards = {(x,y):{dirs_letters[v]} for y,l in enumerate(lines[1:-1]) for x,v in enumerate(l[1:-1]) if v != '.'}
dims = len(lines[0])-2,len(lines)-2

all_blizards = [blizards]
for i in range(1, np.lcm(*dims)):
    new_bliz = defaultdict(set)
    for (x,y),b_dirs in all_bliz[-1].items():
        for dx,dy in b_dirs:
            new_bliz[((x+dx) % dims[0], (y+dy) % dims[1])].add((dx,dy))
    all_blizards.append(new_bliz)

def shortest_path(all_blizards, dims, start_min, start_pos, end_pos):
    cur_nodes = {start_pos}

    for minute in range(start_min+1, 10000):
        next_nodes = {(x,y) 
                      for x in range(dims[0]) for y in range(dims[1])
                      if (x,y) not in all_bliz[minute%len(all_bliz)]}
        next_nodes.add(start_pos)
        
        cur_nodes = {(x+dx,y+dy) for x,y in cur_nodes for dx,dy in dirs
                          if (x+dx,y+dy) in next_nodes }
                    
        if end_pos in cur_nodes:
            # print(f"Reached {n} in {minute} minutes. Solution: {minute+1}")
            return minute+1

t1 = shortest_path(all_blizards, dims=dims, start_min=0, start_pos=(0, -1), end_pos=(dims[0]-1, dims[1]-1))
t2 = shortest_path(all_blizards, dims=dims, start_min=t1, start_pos=(dims[0]-1, dims[1]), end_pos=(0,0))
shortest_path(all_blizards, dims=dims, start_min=t2, start_pos=(0, -1), end_pos=(dims[0]-1, dims[1]-1))


54

In [110]:
# data = """        ...#
#         .#..
#         #...
#         ....
# ...#.......#
# ........#...
# ..#....#....
# ..........#.
#         ...#....
#         .....#..
#         .#......
#         ......#.

# 10R5L5R10L4R5L5"""

# face_dim = 4


In [119]:
grid, insts = data.split("\n\n")

insts = re.split(r"([A-Z])", insts)
grid = {(i,j):v for j,l in enumerate(grid.split('\n')) for i,v in enumerate(l) if v != ' '}


In [136]:
rotations = np.array((
 ((1,   0,  0),
  (0,   0,  -1),
  (0,   1,  0)),
 
 ((0,   0,  1),
  (0,   1,  0),
  (-1,  0,  0)),
 
 ((0,   -1, 0),
  (1,   0,  0),
  (0,   0,  1))
))

faces = {(ii,jj) for ii in range(4) for jj in range(4) if (ii*face_dim, jj*face_dim) in grid}

dirs = ((1,0), (0,1), (-1, 0), (0, -1))
move = lambda p,d: ((p[0]+d[0]), (p[1]+d[1]))

move3d = lambda u, dir: (next(r.T if o == 1 else r
                              for o,r in zip(dir, rotations) 
                              if o != 0) @ u.T).T

def rotate_match(axes, target):
  if (axes == target).all():
    return 0
  for i in range(1, 4):
    axes = move3d(axes, axes[0])
    if (axes == target).all():
      return i
  return False


def search_face(faces, cur_face, axes, target_axes, prev_face=None):
  r = rotate_match(axes, target_axes)
  if r is not False:
    return r, cur_face
  directions = (axes[1]-axes[0], axes[2]-axes[0], axes[0]-axes[1], axes[0]-axes[2])
  for i, dir in enumerate(dirs):
    next_face = tuple(a+b for a,b in zip(cur_face,dir))
    # print(next_face)
    if (next_face == prev_face) or (next_face not in faces):
      continue
    r, face = search_face(faces, next_face, move3d(axes, directions[i]), target_axes, cur_face)
    if r is not False:
      return r, face
  return False, False    

def neighbour(faces, face_from, dir):
  axes = np.array(((0, 0, 1), (1, 0, 1), (0, 1, 1)))
  directions = (axes[1]-axes[0], axes[2]-axes[0], axes[0]-axes[1], axes[0]-axes[2])
  target = move3d(axes, directions[dir])
  return search_face(faces, face_from, axes, target)


print(neighbour(faces, face_from=(2,0), dir=0))
print(neighbour(faces, face_from=(0,1), dir=2))

# (2, (1, 2))
# (1, (0, 3))

[0 0 1]


ValueError: len(axes) must be 2.

In [112]:
# move_m = lambda p,d,b: ((p[0]+d[0])%b, (p[1]+d[1])%b)

# curdir = 0
# curpos = min(p for p in grid if p[1] == 0)

# for i in range(face_dim):
#     for j in range(face_dim):
#         if (i,j) in faces:
#             for d in range(len(dirs)):
#                 if (i+dirs[d][0], j+dirs[d][1]) not in faces:
#                     r, n = neighbour(faces, (i, j), d)
#                     assert n, ((i,j), d)



In [113]:
tiles = {'.': '⬜️', '#': '🟥', ' ': '⬛️'}

def print_grid(grid, curpos=None, curdir=4, bounds_i=None, bounds_j=None):
    if bounds_i is None:
        bounds_i = 0, max(i for i,_ in grid)+1
    if bounds_j is None:
        bounds_j = 0, max(j for _,j in grid)+1
    
    for j in range(*bounds_j):
        for i in range(*bounds_i):
            if curpos is not None and curpos == (i,j):
                if grid.get(curpos, False) != '.':
                    print("?", end="")
                    continue
                    # print("ERROR: curpos not empty:", curpos, grid.get(curpos, False))
                    # print_grid(grid)
                # assert grid.get(curpos) == '.', (curpos, grid.get(curpos), grid)
                print(["⏩", "🔽", "⏪", "🔼", "🤓"][curdir], end="")
                # print("🤓", end="")
            else:
                print(tiles[grid.get((i,j), ' ')], end="")
        print("")
    print("")

In [114]:
rot_ij_90 = lambda i, j: (face_dim-1-j, i)
rot_ij = lambda i, j, r: rot_ij(*rot_ij_90(i, j), r-1) if r > 0 else (i, j)

rot_ij(0, 1, 3)

(1, 49)

In [120]:
move = lambda p,d: ((p[0]+d[0]), (p[1]+d[1]))

# dirs = ((1,0), (0,1), (-1, 0), (0, -1))
curdir = 0
curpos = min(p for p in grid if p[1] == 0)

# print_grid(grid , curpos, curdir)

for i,inst in enumerate(insts):
    if inst in 'RL':
        curdir = (curdir + (1 if inst == "R" else -1)) % 4
    else:
        inst = int(inst)
            
        for _ in range(inst):
            nextpos = move(curpos, dirs[curdir])
            if nextpos not in grid:
                p = (curpos[0]//face_dim, curpos[1]//face_dim)
                r, face = neighbour(faces, p, curdir)
                nextpos = rot_ij(nextpos[0]%face_dim, nextpos[1]%face_dim, 4-r)
                nextpos = (face[0]*face_dim + nextpos[0], face[1]*face_dim + nextpos[1])
                if grid[nextpos] != '#':
                    curdir = (curdir-r) % 4
            if grid[nextpos] == '#':
                break
            curpos = nextpos
    
print(1000*(curpos[1]+1) + 4*(curpos[0]+1) + curdir)
                

100%|██████████| 4001/4001 [00:00<00:00, 22572.68it/s]

137045


In [ ]:
49*18*16

In [ ]:
# 1000000000000 
(1000000000000-37) // 35, (1000000000000-37) % 35

In [ ]:
Cycle starts: 37 bricks | height: 63
full cycle: 72 tot bricks | tot height: 116
cycle brick length: 35


In [ ]:
# min_flows = dict()
# # nx.bfs_tree(G, 'AA', depth_limit=30)
#     # print(t)
# cur_states = {('AA', (), 0)}
# tot_minutes = 30

# for minute in range(1, tot_minutes+1):
#     next_states = set()
#     for cur_node, open_valves, flow in cur_states:
#         flow_rate = sum(flows[n] for n in open_valves)
#         min_flow = flow + flow_rate * (tot_minutes-minute)
#         # print(f"Exploring: {cur_node} {open_valves} {flow}")
#         if min_flows.get(cur_node, -1) >= min_flow:
#             # print(f" -> Pruning: {flow} + {tot_minutes-minute} * {flow_rate} -> {min_flow} <= {min_flows[cur_node]}")
#             continue
#         # print(f" Adding min_flow: {cur_node} -> {min_flow}")
#         min_flows[cur_node] = min_flow
#         flow += flow_rate
#         if (cur_node not in open_valves) and flows[cur_node]:
#             next_states.add((cur_node, tuple(set([*open_valves, cur_node])), flow))
#         for next_node in G.neighbors(cur_node):
#             next_states.add((next_node, open_valves, flow))
#     if len(next_states) == 0:
#         pprint(cur_states)
#         pprint(min_flows)
#     print(f"Minute {minute}: max flow = {max(f for _,_,f in next_states)} ({len(next_states)} active states)")
#     cur_states = next_states
    
#     # pprint(min_flows)

In [ ]:
# sum(v for k,v in flows.items() if k in ('BB', 'DD', 'HH'))

    # ('BB', 'HH') ('DD', 'JJ
# flows

In [ ]:
all_valves = {k for k,v in flows.items() if v}
min_flows = dict()
# nx.bfs_tree(G, 'AA', depth_limit=30)
    # print(t)
cur_states = {(('AA', 'AA'), (), 0, ())}
tot_minutes = 26

for minute in range(1, tot_minutes+1):
    print(f"Minute {minute}: max flow = {max([0, *min_flows.values()])} ({len(cur_states)} active states)\n")
        
    next_states = set()
    for cur_nodes, open_valves, flow, prev_nodes in  sorted(cur_states, key=lambda x: x[2], reverse=True):
        flow_rate = sum(flows[n] for n in open_valves)
        flow += flow_rate
        
        min_flow = flow + flow_rate * (tot_minutes-minute)

        if min_flows.get(cur_nodes, -1) >= min_flow:
            continue
        
        min_flows[cur_nodes] = min_flow
        
        if len(open_valves) == len(all_valves):
            continue
        
            
        next_states_n = [[], []]
        for i,cur_node in enumerate(cur_nodes):
            next_states_n[i] = [([(next_node, []) 
                                 for next_node in G.neighbors(cur_node)
                                 if next_node not in (prev_nodes)]
                                + ([(cur_node, [cur_node])] if flows[cur_node] and cur_node not in open_valves else []))
                                
            ]
        debug = {(
            tuple(sorted([cur_node1, cur_node2])), 
            tuple(sorted(set([*valve1, *valve2, *open_valves]))), 
            flow,
            (cur_nodes)
        )
                            for cur_node1, valve1 in next_states_n[0]
                            for cur_node2, valve2 in next_states_n[1]
                            }

        next_states.update(debug)
        
    cur_states = next_states

    if len(cur_states) == 0:
        print(f"No more moves.")
        break
    
print(f" Max flow: {max(min_flows.values())}")
    # pprint(min_flows)
    # print(max(min_flows.values()), '\n')
    # 2532
    # 2349
    # 2715
    
    # ('BB', 'HH') ('DD', 'JJ

In [ ]:
from aocd import get_data, submit
import numpy as np
import sys
import itertools
from collections import defaultdict, Counter
import re

day = 18
year = 2022

data = get_data(day=day, year=year)


In [ ]:
cubes = {tuple([int(i) for i in l.split(',')]) for l in data.split()}

faces = len(cubes) * 6
faces -= sum( tuple(cc + d*pp for cc,pp in zip(c,p)) in cubes
             for c in cubes for d in (-1, 1) for p in ((1,0,0),(0,1,0),(0,0,1)))

print("Part 1:", faces)


In [ ]:
m = max(max(cubes, key=lambda x: x[i])[i] for i in range(3))

steam = set(
    [(a, b, c) for a in (-1,m+2) for b in range(-1, m+2) for c in range(-1, m+2)]
+   [(b, c, a) for a in (-1,m+2) for b in range(-1, m+2) for c in range(-1, m+2)]
+   [(c, a, b) for a in (-1,m+2) for b in range(-1, m+2) for c in range(-1, m+2)]
)

expanded = True
while expanded:
    expanded = False
    for x in range(0, m+1):
        for y in range(0, m+1):
            for z in range(0, m+1):
                if (x,y,z) in cubes or (x,y, z) in steam:
                    continue
                for d in (-1,1):
                    if (x+d,y,z) in steam or (x,y+d,z) in steam or (x,y,z+d) in steam:
                        steam.add((x,y,z))
                        expanded = True
                        break

cooling = {(x,y,z)
    for x in range(0, m+1)
        for y in range(0, m+1)
            for z in range(0, m+1)
                if (x,y,z) not in steam }

faces = len(cubes) * 6
faces -= sum( tuple(cc + d*pp for cc,pp in zip(c,p)) in cooling
             for c in cubes for d in (-1, 1) for p in ((1,0,0),(0,1,0),(0,0,1)))


print("Part 2:", faces)